In [1]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys
from pyspark.sql import SparkSession
import os
import gdown
import pandas as pd
from pyspark import SparkFiles
from pyspark.sql import functions as F


In [3]:
spark = SparkSession.builder \
.master("local") \
.appName("Exercise2") \
.getOrCreate()

In [4]:
csvPath = 'exampleData.csv'
if not os.path.exists(csvPath):
    gdown.download(id = "1YLo060ccO-JNBJ7Cn_DT_y0LTkps-0Lg", output=csvPath, quiet=False)

In [5]:
df = spark.read.csv("exampleData.csv", header=True, inferSchema=True)

In [6]:
staticSchema = df.schema

Reading the schema

In [7]:
streamingDF = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("/home/jovyan/*.csv")

In [8]:
streamingDF.isStreaming

True

In [9]:
filteredDF = streamingDF.filter(F.col('air_temperature')<5)

In [14]:
'''append, update, complete'''
df = filteredDF.writeStream.format("memory")\
.queryName("weatherQuery") \
.outputMode("append") \
.start()

'append, update, complete'

In [15]:
df.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [17]:
spark.sql("""
select *
FROM weatherQuery
""").show(5)

+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|_c0|         dateTime|indicator_rain|precipitation|indicator_temp|air_temperature|indicator_wetb|wetb|dewpt|vappr|relative_humidity|   msl|indicator_wdsp|wind_speed|indicator_wddir|wind_from_direction|
+---+-----------------+--------------+-------------+--------------+---------------+--------------+----+-----+-----+-----------------+------+--------------+----------+---------------+-------------------+
|298|12-jan-1990 20:00|             3|          0.0|             0|            4.9|             0| 4.3|  3.4|  7.8|               90|1025.5|             2|         1|              2|                210|
|300|12-jan-1990 22:00|             3|          0.0|             0|            3.9|             0| 3.2|  2.1|  7.1|               88|1025.9|             2|         1|              2|      